**Reading from the Bronze table**

In [0]:
bronze_table_path = '/Volumes/workspace/default/my_volume/Sentiment_Project/bronze_tweets/'

bronze_df = spark.read.format('delta').load(bronze_table_path)
print(f"Total records: {bronze_df.count()}")
display(bronze_df.limit(10))

**Data Cleaning**

In [0]:
from pyspark.sql.functions import regexp_replace, trim, lower, col

silver_df = bronze_df.withColumn(
    'Cleaned_Text',
    regexp_replace(
        lower(
            regexp_replace(
                col('Sentence'),
                '[^a-zA-Z0-9\\s]', # --> Removing non alphanumeric characters (keeping only letters, numbers, and spaces)
                ' ' # Replacing with a space
                )
        ),
        'https\\s+', # --> Removing URLs
        ''
    )
).withColumn(
    'Cleaned_Text',
    regexp_replace(
        trim(col('Cleaned_Text')), # --> Removing leading and trailing spaces
        '\\s+', # --> Replacing multiple spaces with a single space
        ' '
    )
)

**Display cleaned data**

In [0]:
display(silver_df.limit(10).select('Sentence','Cleaned_Text','Sentiment'))

**Writing the cleaned data to the _Silver Table_**

In [0]:
silver_table_path = '/Volumes/workspace/default/my_volume/Sentiment_Project/silver_tweets_cleaned'

silver_df.write.format('delta').mode('overwrite').save(silver_table_path)

**Verifying silver _Table_**

In [0]:
silver_verify_df = spark.read.format('delta').load(silver_table_path)
print(f"Total records in the silver table: {silver_verify_df.count()}")
display(silver_verify_df.select('Sentence','Cleaned_Text','Sentiment').limit(10))